## Treinando milhares de modelos preditivos simultaneamente com hiperparametros
### Vou criar e treinar mais de 7.000 modelos preditivos com 5 algoritmos diferentes

### Random Forest, SVM, KNN, Naive Bayes e Regressão Logistica


In [2]:
# Importar os pacotes que irei utilizar.

import pandas as pd
import numpy as np
import time

import warnings
warnings.filterwarnings("ignore") 


from sklearn.svm import SVC
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 50)

## Importação dos dados, Analise Exploratória e Tratamento de Dados

In [5]:
# Comando utilizado para carregar o arquivo e armazena-lo como um DataFrame do Pandas

df_dados = pd.read_excel("dados.xlsx")

In [6]:
# Comando utilizado para verificar a quantidade de linhas e colunas do arquivo
# Colunas também são chamadas de variáveis.

df_dados.shape

(2000, 11)

In [9]:
# Comando utilizado para verificar informações sobre os dados(Tipo de variáveis, Variáveis, Quantidade de registros, etc)

df_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   UF                          2000 non-null   object 
 1   IDADE                       2000 non-null   int64  
 2   ESCOLARIDADE                2000 non-null   object 
 3   ESTADO_CIVIL                2000 non-null   object 
 4   QT_FILHOS                   2000 non-null   int64  
 5   VL_IMOVEIS                  2000 non-null   int64  
 6   TEMPO_ULTIMO_EMPREGO_MESES  2000 non-null   int64  
 7   ULTIMO_SALARIO              2000 non-null   int64  
 8   VALOR_TABELA_CARROS         2000 non-null   int64  
 9   SCORE_CREDITO               2000 non-null   float64
 10  EMPRESTIMO                  2000 non-null   object 
dtypes: float64(1), int64(6), object(4)
memory usage: 172.0+ KB


Como podemos perceber acima, temos 3 variáveis OBJECT e precisamos fazer o ONEHOTENCODING nelas.

In [12]:
# Comando utilizado para avaliar se alguma variável possui valor nulo ou chamado de valores missing ou NAN (Not Available)

df_dados.isnull().sum()

UF                            0
IDADE                         0
ESCOLARIDADE                  0
ESTADO_CIVIL                  0
QT_FILHOS                     0
VL_IMOVEIS                    0
TEMPO_ULTIMO_EMPREGO_MESES    0
ULTIMO_SALARIO                0
VALOR_TABELA_CARROS           0
SCORE_CREDITO                 0
EMPRESTIMO                    0
dtype: int64

## Pré Processamento dos Dados

In [15]:
# Cria o encoder
lb = LabelEncoder()

# Aplica o encoder nas variáveis que estão com string
df_dados['ESTADO_CIVIL'] = lb.fit_transform(df_dados['ESTADO_CIVIL'])
df_dados['ESCOLARIDADE'] = lb.fit_transform(df_dados['ESCOLARIDADE'])
df_dados['UF'] = lb.fit_transform(df_dados['UF'])

In [17]:
df_dados.head()

,UF,IDADE,ESCOLARIDADE,ESTADO_CIVIL,QT_FILHOS,VL_IMOVEIS,TEMPO_ULTIMO_EMPREGO_MESES,ULTIMO_SALARIO,VALOR_TABELA_CARROS,SCORE_CREDITO,EMPRESTIMO
0,4,19,2,2,0,0,8,1800,0,56.000000,SIM
1,0,23,1,2,1,0,9,4800,50000,18.000000,NAO
2,3,25,0,0,0,220000,18,2200,30000,45.000000,SIM
3,1,27,2,0,1,0,22,3900,0,28.666667,NAO
4,4,30,0,1,0,0,14,3100,40000,39.666667,SIM


In [19]:
# Separando a variavel alvo

target = df_dados.iloc[:,10]

In [20]:
target.head()

0    SIM
1    NAO
2    SIM
3    NAO
4    SIM
Name: EMPRESTIMO, dtype: object

In [23]:
# Separando as variaveis preditoras

preditoras = df_dados.copy() #Fazendo uma cópia do dataframe

del preditoras['EMPRESTIMO'] #Excluindo a variavel target, pois já separei ela na etapa anterior

preditoras.head() #Visualizando as variaveis preditoras

,UF,IDADE,ESCOLARIDADE,ESTADO_CIVIL,QT_FILHOS,VL_IMOVEIS,TEMPO_ULTIMO_EMPREGO_MESES,ULTIMO_SALARIO,VALOR_TABELA_CARROS,SCORE_CREDITO
0,4,19,2,2,0,0,8,1800,0,56.000000
1,0,23,1,2,1,0,9,4800,50000,18.000000
2,3,25,0,0,0,220000,18,2200,30000,45.000000
3,1,27,2,0,1,0,22,3900,0,28.666667
4,4,30,0,1,0,0,14,3100,40000,39.666667


In [25]:
# Divisão em Dados de Treino e Teste.

X_treino, X_teste, y_treino, y_teste = train_test_split(preditoras, target, test_size = 0.3, random_state = 40)

In [27]:
# Vamos aplicar a normalização em treino e teste

sc = MinMaxScaler()
X_treino_normalizados = sc.fit_transform(X_treino)
X_teste_normalizados = sc.transform(X_teste)

## Algoritmo Random Forest

In [62]:
# Construtor do Modelo
# Crie o modelo com parâmetros explícitos

randomForest = RandomForestClassifier(
    bootstrap=True,  # Valor padrão, mas será sobrescrito pelo GridSearchCV
    random_state=42
)

In [64]:
# Visualiza os Hiperparametros padrão do Modelo

print(randomForest.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [100]:
# Valores para o grid de hiperparametros

n_estimators = np.array([100, 150, 200, 250, 300])
max_depth = np.array([10, 20])
criterion = np.array(["gini", "entropy"])
max_features = np.array(["sqrt", "log2", None])
min_samples_split = np.array([2, 5, 10])  
min_samples_leaf = np.array([1, 2, 3])    
bootstrap = np.array([True, False])      
 

In [82]:
# Grid de hiperparâmetros

grid_parametros = dict(
    n_estimators=n_estimators,
    max_depth=max_depth,
    criterion=criterion,
    max_features=max_features,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    bootstrap=bootstrap  
)

In [84]:
# Criando o modelo com o Grid de Hiperparametros

randomForest_grid = GridSearchCV(
    randomForest,
    grid_parametros,
    cv=3,
    n_jobs=8,
    error_score='raise'  # Para debugar melhor se houver erros
)

In [86]:
# Treinando os modelos

inicio = time.time()
randomForest_grid.fit(X_treino_normalizados, y_treino)
fim = time.time()

In [92]:
# Obtendo e visualizando os parametros treinados
treinos_rf = pd.DataFrame(randomForest_grid.cv_results_)

# Acurácia em Treino
print(f"Acurácia em Treinamento: {randomForest_grid.best_score_ :.2%}")
print("")
print(f"Hiperparâmetros Ideais: {randomForest_grid.best_params_}")
print("")
print("Tempo de Treinamento do Modelo: ", round(fim - inicio,2))
print("")
print("Numero de treinamentos realizados: ", treinos_rf.shape[0])

Acurácia em Treinamento: 78.64%

Hiperparâmetros Ideais: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 20, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 150}

Tempo de Treinamento do Modelo:  174.21

Numero de treinamentos realizados:  1080


In [94]:
# Dicionário de métricas e metadados

modelo_rf = {'Melhores Hiperparametros':randomForest_grid.best_params_,
               'Numero de Modelos Treinados': treinos_rf.shape[0],  
               'Melhor Score': str(round(randomForest_grid.best_score_ * 100,2))+"%"}

In [96]:
modelo_rf

{'Melhores Hiperparametros': {'bootstrap': True,
  'criterion': 'entropy',
  'max_depth': 20,
  'max_features': None,
  'min_samples_leaf': 1,
  'min_samples_split': 5,
  'n_estimators': 150},
 'Numero de Modelos Treinados': 1080,
 'Melhor Score': '78.64%'}

## Algoritmo SVM (Suport Vector Machine)

In [102]:
modelo_svm = SVC()

In [104]:
modelo_svm.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [106]:
# Valores para o grid de hiperparametros
grid_parametros = {'C': [0.1,1,10,100],
                   'gamma': [1,0.1,0.01,0.001],
                   'kernel': ['poly','rbf','sigmoid','linear'],
                   'degree' : [2,3,4,],
                   'coef0' : [0.5,1],
                   'decision_function_shape':['ovo', 'ovr'],
                   'max_iter': [-1, 1]}


svm = GridSearchCV(modelo_svm, grid_parametros, n_jobs = 8)

# Treinando os modelos
inicio = time.time()
svm.fit(X_treino_normalizados, y_treino)
fim = time.time()

# Obtendo e visualizando os parametros treinados
treinos_svm = pd.DataFrame(svm.cv_results_)

# Acurácia em Treino
print(f"Acurácia em Treinamento: {svm.best_score_ :.2%}")
print("")
print(f"Hiperparâmetros Ideais: {svm.best_params_}")
print("")
print("Tempo de Treinamento do Modelo: ", round(fim - inicio,2))
print("")
print("Numero de treinamentos realizados: ", treinos_svm.shape[0])

Acurácia em Treinamento: 75.07%

Hiperparâmetros Ideais: {'C': 10, 'coef0': 0.5, 'decision_function_shape': 'ovo', 'degree': 4, 'gamma': 1, 'kernel': 'poly', 'max_iter': -1}

Tempo de Treinamento do Modelo:  44.86

Numero de treinamentos realizados:  1536


In [108]:
# Dicionário de métricas e metadados

modelo_svm = {'Melhores Hiperparametros':svm.best_params_,
               'Numero de Modelos Treinados': treinos_svm.shape[0],  
               'Melhor Score': str(round(svm.best_score_ * 100,2))+"%"}

In [110]:
modelo_svm

{'Melhores Hiperparametros': {'C': 10,
  'coef0': 0.5,
  'decision_function_shape': 'ovo',
  'degree': 4,
  'gamma': 1,
  'kernel': 'poly',
  'max_iter': -1},
 'Numero de Modelos Treinados': 1536,
 'Melhor Score': '75.07%'}

## Algoritmo Naive Bayes

In [112]:
nb = CategoricalNB()

In [114]:
nb.get_params()

{'alpha': 1.0,
 'class_prior': None,
 'fit_prior': True,
 'force_alpha': True,
 'min_categories': None}

In [158]:
# 1. Verificação robusta dos dados de entrada
print("\nVerificando dados de entrada...")

# Converter para numpy array se não for
if not isinstance(X_treino_normalizados, np.ndarray):
    X_treino_normalizados = np.array(X_treino_normalizados)
if not isinstance(y_treino, np.ndarray):
    y_treino = np.array(y_treino)

print(f"Tipo de X_treino: {type(X_treino_normalizados)}")
print(f"Forma de X_treino: {X_treino_normalizados.shape}")
print(f"Tipo de y_treino: {type(y_treino)}")
print(f"Forma de y_treino: {y_treino.shape}")

# 2. Modelo base
nb = CategoricalNB()

# 3. Grid de parâmetros simplificado e validado
param_grid = {
    'alpha': [0.1, 0.5, 1.0],  # Valores float positivos
    'fit_prior': [True, False],  # Booleanos reais
    'min_categories': [1, 2]     # Inteiros positivos
}

# 4. Configuração do GridSearchCV
nb_grid = GridSearchCV(
    estimator=nb,
    param_grid=param_grid,
    cv=3,
    n_jobs=1,  # Mantido como 1 para debug
    error_score='raise',
    verbose=3
)

# 5. Treinamento com tratamento de erros robusto
print("\nIniciando treinamento...")
inicio = time.time()

try:
    nb_grid.fit(X_treino_normalizados, y_treino)
    fim = time.time()
    
    print("\nTreinamento concluído com sucesso!")
    print(f"Melhor acurácia: {nb_grid.best_score_:.2%}")
    print(f"Melhores parâmetros: {nb_grid.best_params_}")
    print(f"Tempo total: {fim - inicio:.2f} segundos")

except Exception as e:
    print("\nFalha no treinamento:")
    print(f"Tipo do erro: {type(e).__name__}")
    print(f"Mensagem: {str(e)}")
    
    # Informações adicionais para debug
    print("\nInformações adicionais:")
    print("Tipos únicos em X_treino:", np.unique(X_treino_normalizados))
    print("Classes em y_treino:", np.unique(y_treino))
    print("Tempo de Treinamento do Modelo: ", round(fim - inicio,2))
    print("")
    print("Numero de treinamentos realizados: ", nb_grid.shape[0])
    
    raise


Verificando dados de entrada...
Tipo de X_treino: <class 'numpy.ndarray'>
Forma de X_treino: (1400, 10)
Tipo de y_treino: <class 'numpy.ndarray'>
Forma de y_treino: (1400,)

Iniciando treinamento...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV 1/3] END alpha=0.1, fit_prior=True, min_categories=1;, score=0.527 total time=   0.0s
[CV 2/3] END alpha=0.1, fit_prior=True, min_categories=1;, score=0.495 total time=   0.0s
[CV 3/3] END alpha=0.1, fit_prior=True, min_categories=1;, score=0.552 total time=   0.0s
[CV 1/3] END alpha=0.1, fit_prior=True, min_categories=2;, score=0.527 total time=   0.0s
[CV 2/3] END alpha=0.1, fit_prior=True, min_categories=2;, score=0.495 total time=   0.0s
[CV 3/3] END alpha=0.1, fit_prior=True, min_categories=2;, score=0.552 total time=   0.0s
[CV 1/3] END alpha=0.1, fit_prior=False, min_categories=1;, score=0.525 total time=   0.0s
[CV 2/3] END alpha=0.1, fit_prior=False, min_categories=1;, score=0.507 total time=   0.0s
[CV 3/3] END alph

In [160]:
# Dicionário de métricas e metadados - USANDO O OBJETO GRIDSEARCH CORRETO

modelo_naiveBayes = {
    'Melhores Hiperparametros': nb_grid.best_params_,
    'Numero de Modelos Treinados': len(nb_grid.cv_results_['params']),  
    'Melhor Score': f"{round(nb_grid.best_score_ * 100, 2)}%"
}

In [146]:
modelo_naiveBayes

{'Melhores Hiperparametros': {'alpha': 0.1,
  'fit_prior': False,
  'min_categories': 1},
 'Numero de Modelos Treinados': 12,
 'Melhor Score': '52.79%'}

## Modelo KNN (K Nearest Neighbors)

In [148]:
knn = KNeighborsClassifier()

In [150]:
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [152]:
# Valores para o grid de hiperparametros
n_neighbors = np.array([3,4,5,6,7,8,9,10,11,12,13,14,15])
algorithm = np.array(['auto', 'ball_tree', 'kd_tree', 'brute'])
leaf_size = np.array([30,31,32,33,34,35,36,37,38,39,40])
metric =  np.array(['minkowski','euclidean'])


# Grid de hiperparâmetros
grid_parametros = dict(n_neighbors = n_neighbors, algorithm = algorithm, leaf_size = leaf_size,metric = metric)

knn = GridSearchCV(knn, grid_parametros, n_jobs = 8)

# Treinando os modelos
inicio = time.time()
knn.fit(X_treino_normalizados, y_treino)
fim = time.time()

# Obtendo e visualizando os parametros treinados
treinos_knn = pd.DataFrame(knn.cv_results_)

# Acurácia em Treino
print(f"Acurácia em Treinamento: {knn.best_score_ :.2%}")
print("")
print(f"Hiperparâmetros Ideais: {knn.best_params_}")
print("")
print("Tempo de Treinamento do Modelo: ", round(fim - inicio,2))
print("")
print("Numero de treinamentos realizados: ", treinos_knn.shape[0])

Acurácia em Treinamento: 74.14%

Hiperparâmetros Ideais: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'n_neighbors': 3}

Tempo de Treinamento do Modelo:  14.03

Numero de treinamentos realizados:  1144


In [154]:
# Dicionário de métricas e metadados

modelo_knn = {'Melhores Hiperparametros':knn.best_params_,
               'Numero de Modelos Treinados': treinos_knn.shape[0],  
               'Melhor Score': str(round(knn.best_score_ * 100,2))+"%"}

In [156]:
modelo_knn

{'Melhores Hiperparametros': {'algorithm': 'auto',
  'leaf_size': 30,
  'metric': 'minkowski',
  'n_neighbors': 3},
 'Numero de Modelos Treinados': 1144,
 'Melhor Score': '74.14%'}

## Modelo Logistic Regression  

In [162]:
regressaoLogistica = LogisticRegression()

In [164]:
regressaoLogistica.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [166]:
# Valores para o grid de parametros
max_iter = np.array([100,110,120,130,140,150])
dual = np.array(['True','False'])
c = np.array([1.0,1.5,1.8,2.0])
tol = np.array([0.0001,0.001,0.01,0.1, 1])
penalty =  np.array(['l1','l2'])
intercept_scaling = np.array([1.0,2.0])
solver = np.array(['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])

# Grid de hiperparâmetros
grid_parametros = dict(max_iter = max_iter,
                       C = c,
                       penalty = penalty,
                       tol = tol,
                       solver = solver,
                       intercept_scaling = intercept_scaling)

regressaoLogistica = GridSearchCV(regressaoLogistica, grid_parametros, n_jobs = 8)

# Treinando os modelos
inicio = time.time()
regressaoLogistica.fit(X_treino_normalizados, y_treino)
fim = time.time()

# Obtendo e visualizando os parametros treinados
treinos_rlog = pd.DataFrame(regressaoLogistica.cv_results_)

# Acurácia em Treino
print(f"Acurácia em Treinamento: {regressaoLogistica.best_score_ :.2%}")
print("")
print(f"Hiperparâmetros Ideais: {regressaoLogistica.best_params_}")
print("")
print("Tempo de Treinamento do Modelo: ", round(fim - inicio,2))
print("")
print("Numero de treinamentos realizados: ", treinos_rlog.shape[0])

Acurácia em Treinamento: 66.50%

Hiperparâmetros Ideais: {'C': 1.0, 'intercept_scaling': 1.0, 'max_iter': 110, 'penalty': 'l2', 'solver': 'sag', 'tol': 1.0}

Tempo de Treinamento do Modelo:  5.59

Numero de treinamentos realizados:  2400


In [168]:
# Dicionário de métricas e metadados

modelo_regressaoLogistica = {'Melhores Hiperparametros':regressaoLogistica.best_params_,
               'Numero de Modelos Treinados': treinos_rlog.shape[0],  
               'Melhor Score': str(round(regressaoLogistica.best_score_ * 100,2))+"%"}

In [170]:
modelo_regressaoLogistica

{'Melhores Hiperparametros': {'C': 1.0,
  'intercept_scaling': 1.0,
  'max_iter': 110,
  'penalty': 'l2',
  'solver': 'sag',
  'tol': 1.0},
 'Numero de Modelos Treinados': 2400,
 'Melhor Score': '66.5%'}

In [172]:
# Gerando o DataFrame com todos os valores de todos os modelos treinados

resumo = pd.DataFrame({'Random Forest':pd.Series(modelo_rf),
                       'SVM':pd.Series(modelo_svm),
                       'Naive Bayes':pd.Series(modelo_naiveBayes),
                       'KNN':pd.Series(modelo_knn),
                       'Regressão Logistica':pd.Series(modelo_regressaoLogistica)})

In [174]:
resumo

,Random Forest,SVM,Naive Bayes,KNN,Regressão Logistica
Melhores Hiperparametros,"{'bootstrap': True, 'criterion': 'entropy', 'm...","{'C': 10, 'coef0': 0.5, 'decision_function_sha...","{'alpha': 0.1, 'fit_prior': False, 'min_catego...","{'algorithm': 'auto', 'leaf_size': 30, 'metric...","{'C': 1.0, 'intercept_scaling': 1.0, 'max_iter..."
Numero de Modelos Treinados,1080,1536,12,1144,2400
Melhor Score,78.64%,75.07%,52.79%,74.14%,66.5%


## Análise Prática dos Resultados

### 1. Comparação de Desempenho

- Random Forest (78.64%) claramente teve o melhor desempenho entre os modelos testados

- SVM (75.07%) e KNN (74.14%) tiveram desempenhos próximos

- Regressão Logística (66.5%) e Naive Bayes (52.79%) ficaram significativamente atrás

Insight Prático: De acordo com o que foi apresentado o ideal de implementação seria o Random Forest em produção, mas manteria o SVM como alternativa por ter desempenho similar.

### 2. Custo-Complexidade vs. Performance

- Random Forest teve 1080 combinações testadas (alto custo computacional)

- Naive Bayes só testou 12 combinações (baixo custo) mas performance ruim

- SVM com bom desempenho mas 1536 combinações testadas (alto custo)

Insight Prático: Para problemas que precisam de retreinamento frequente, o KNN (1144 combinações) pode ser uma opção mais viável que o SVM, com desempenho similar.

### 3. Interpretabilidade dos Modelos

- Regressão Logística (pior desempenho) é mais interpretável

- Random Forest (melhor desempenho) é menos interpretável

Ação Prática: Se a interpretabilidade for crucial (como em crédito ou saúde), pode-se sacrificar performance pela Regressão Logística. Caso contrário, opta-se pelo Random Forest.

## Como Isso é Usado na Vida Real

### 1. Tomada de Decisão de Implementação

Eu sendo o analista criaria um relatório recomendando:

- Implementar o Random Forest como modelo principal

- Manter o SVM como modelo secundário para comparação

- Descartar Naive Bayes por desempenho insuficiente